In [1]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
import numpy as np
import math

# Data reading + EDA

In [2]:
# path = os.getcwd() + r'/data/'
# aisles = pd.read_csv(path + r'/aisles.csv', index_col=0)
# departments = pd.read_csv(path + r'/departments.csv', index_col=0)
# products = pd.read_csv(path + r'/products.csv')
# orders = pd.read_csv(path + r'/orders.csv')
# order_product_prior = pd.read_csv(path + r'/order_products__prior.csv')
# order_product_train = pd.read_csv(path + r'/order_products__train.csv')

In [3]:
# """
# Data preparation, since feedback is implicit, we can use amount of times user bought the product,
# that's why we need to have a split before the rating evaluation.

# Split is taken by last order each user've done, since everyone had been done at least 3 orders

# """

# order_product = pd.concat([order_product_prior, order_product_train])
# products = products.merge(departments, on='department_id')
# products = products.merge(aisles, on='aisle_id')
# products.drop(['aisle_id', 'department_id'], axis=1, inplace=True)
# orders = orders.set_index('order_id')
# order_product = order_product.merge(products, on='product_id')
# merged_data = order_product.merge(orders, on='order_id')
# merged_data = merged_data.drop(['add_to_cart_order', 'order_dow', 'days_since_prior_order', 'eval_set'], axis=1)

# data = merged_data
# last_orders = data.order_id.isin(
#             data.sort_values(by=['user_id', 'order_number']).groupby('user_id').tail(1)['order_id'])

# train_data=data[~last_orders]
# test_data=data[last_orders]



# ratings = train_data.groupby(['user_id', 'product_id']).count()['order_id']
# ratings.name = 'amount'
# train_data = train_data.merge(ratings, on=['user_id', 'product_id'])

In [4]:
# train_data.to_csv('train_data.csv')
# test_data.to_csv('test_data.csv')

In [5]:
import pandas as pd
import os
from scipy.sparse import csr_matrix

In [6]:
train_data = pd.read_csv('train_data.csv', index_col=0)
test_data = pd.read_csv('test_data.csv', index_col=0)

C:\Users\user\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
test_data.order_id.nunique()

206209

## EDA

In [10]:
data[['product_id', 'user_id', 'order_id']].nunique()

In [ ]:
data.groupby('user_id')['order_id'].count().sort_values()[::-1]

In [ ]:
data.groupby('user_id')['order_id'].count().sort_values().hist()

In [ ]:
data.groupby('product_id')['order_id'].count().sort_values()

In [ ]:
tmp = data.groupby('product_id')['order_id'].count().sort_values()
tmp[tmp < 1000].hist()
del tmp

Существуют выбросы в большую сторону, например, самые популярные продукты, как бананы. Большинство товаров заказано меньше 100 раз

Минимум 3 заказа на пользователя

Минимум 1 заказ на продукт


# Метрики (из первого задания)

In [170]:
def hitrate(y_true, y_pred, k):
    return np.mean([len([x for x in y_pred[i][:k] if x in y_true[i]]) > 0 for i in range(len(y_pred))])

In [171]:
def p(y_true, y_pred, k):
    return len([x for x in y_pred[:k] if x in y_true])/k
def ap(y_true, y_pred, k):
    if k > len(y_pred):
        print(y_pred, len(y_pred))
        raise(ValueError)
    if k == 0:
        return 0
    tmp = np.sum([int(y_pred[i] in y_true) for i in range(k)])
    if tmp == 0:
        return 0
    return sum([(y_pred[i] in y_true) * p(y_true, y_pred, i+1) for i in range(k)]) / tmp
def map_k(y_true, y_pred, k):
    try:
        return np.mean([ap(y_true[i], y_pred[i], k) for i in range(len(y_true))])
    except ValueError:
        print(i)
        return 0

In [172]:
def dcg(ratings, k):
    if k > len(ratings):
        raise ValueError
    return np.sum([(2**ratings[i] - 1) / math.log2(i+2) for i in range(k)])
def idcg(k):
    return np.sum([1 / math.log2(i+2) for i in range(k)])
def ndcg(ratings, k):
    if k > len(ratings):
        raise ValueError
    return dcg(ratings, k) / idcg(k)
def mean_ndcg(results, k):
    return np.mean([ndcg(i, k) for i in results])

def evaluate_ndcg(data, k):
    return data.groupby('user_id')['amount'].apply(list).apply(lambda x: ndcg(x, 10)).mean()

In [178]:
"""Calculate all the metrics at once"""

def metrics(test, result, k=10):
    res = result.sort_values(by=['user_id', 'item_order'])
#     res = res.drop_duplicates(['user_id', 'product_id'])
    test = test.sort_values(by='user_id')
    y_true = list(test.groupby('user_id')['product_id'].apply(list))
    preds = list(res.groupby('user_id')['product_id'].apply(list))
    print('Hitrate@{} '.format(k), round(hitrate(y_true, preds, k), 3))
    print('MAP@{} '.format(k), round(map_k(y_true, preds, k), 3))
    print('nDCG@{} '.format(k), )

## CF

In [12]:
train_data = train_data[['user_id', 'product_id', 'order_id', 'amount', 'order_number']]
test_data = test_data[['user_id', 'product_id', 'order_id']]

In [13]:
train_data

,user_id,product_id,order_id,amount,order_number
0,202279,33120,2,5,3
1,202279,33120,104690,5,8
2,202279,33120,132412,5,6
3,202279,33120,2808715,5,5
4,202279,33120,2894949,5,1
...,...,...,...,...,...
31653684,117521,13189,2858225,1,3
31653685,189140,47680,1666202,1,2
31653686,182854,34390,1699504,1,10
31653687,51836,31530,2666238,1,12


Creating maps to encode users/items in scipy sparse matrix to avoid memory issues in further processing

In [14]:
user_map = {v: i for i,v in enumerate(sorted(train_data['user_id'].unique()))}

In [15]:
item_map = {v: i for i,v in enumerate(sorted(train_data['product_id'].unique()))}

In [16]:
"""Creates CF matrix based on the amount of buys per product, drop duplicates to avoid squaring the rating"""

CF  = csr_matrix((train_data.amount, 
                  (train_data.user_id.apply(lambda x: user_map[x]),
                   train_data.product_id.apply(lambda x: item_map[x]))))

In [17]:
CF.shape

(206209, 49668)

# Recommenders

## Most popular

In [24]:
MP = train_data.groupby('product_id').count().sort_values(by='user_id', ascending=False)[:20].index

In [25]:
MP

Int64Index([24852, 13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209,
            27845, 27966, 22935, 24964, 45007, 39275, 49683, 28204,  5876,
             8277, 40706],
           dtype='int64', name='product_id')

In [26]:
k = 10
result = pd.DataFrame({
            'user_id': np.repeat(test_data.user_id.unique(), k),
            'product_id': list(MP[:k]) * test_data.user_id.unique().shape[0],
            'item_order': list(range(1, k + 1)) * test_data.user_id.unique().shape[0]
        })

In [140]:
MP_res = result

In [141]:
"""Merge result on train to get the rating of the predicted products"""

MP_res = MP_res.merge(train_data.drop_duplicates(['user_id', 'product_id']), on=['user_id', 'product_id'], how='left')[['user_id', 'product_id', 'item_order', 'amount']]

In [142]:
MP_res

,user_id,product_id,item_order,amount
0,116952,24852,1,6.0
1,116952,13176,2,NaN
2,116952,21137,3,6.0
3,116952,21903,4,NaN
4,116952,47209,5,NaN
...,...,...,...,...
2062085,184060,47766,6,NaN
2062086,184060,47626,7,NaN
2062087,184060,16797,8,NaN
2062088,184060,26209,9,NaN


In [143]:
MP_res['amount'][MP_res.amount.isna()] = 0

In [144]:
# %%time
# metrics(test_data, MP_res)

In [145]:
MP_res.amount = MP_res.amount / max(MP_res.amount) * 3

In [146]:
MP_res.groupby('user_id')['amount'].apply(list).apply(lambda x: ndcg(x, 10)).mean()

0.03419433657726226

#### Results of MP (just to safe and don't rerun code)
- Hitrate@10  0.457
- MAP@10  0.22
- nDCG@10 0.034

## SVD recommender

In [151]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

In [152]:
"""Uses SVD decompositions to create user/items embeddings to recreate matrix afterwards"""

svd = TruncatedSVD(n_components=100, random_state=42)

u = svd.fit_transform(CF) / svd.singular_values_
s = np.diag(svd.singular_values_)
vt = svd.components_

In [153]:
u.shape

(206209, 100)

In [154]:
"""Batch processing to avoid memory issues"""

def batch(obj, n):
    lenght = len(obj)
    for i in range(0, lenght, n):
        yield obj[i:min(i + n, lenght)]

In [155]:
from tqdm import tqdm

In [156]:

"""Recreates matrix by batches, sort values, gets recommendations for each user. Type int32 to reduce memory usage,
since ratings are 

"""
k = 10
top_matrix_ind = []
for b in tqdm(batch(u, 50)):
    top_matrix_ind += list(
                map(lambda x: np.array(x[:k], dtype='int32'),
                    np.argsort(-np.dot(np.dot(b, s), vt))))

4125it [21:20,  3.22it/s]


In [158]:
"""Decode recommendations from CF maps"""

rev_item_map = {v: k for k, v in item_map.items()}
svd_res = np.array([list(map(lambda x: rev_item_map[x], arr)) for arr in top_matrix_ind])


In [159]:
svd_res = [svd_res[i] for i in test_data.user_id.apply(lambda x: user_map[x]).unique()]

In [160]:
SVD_res = pd.DataFrame(np.repeat(test_data.user_id.unique(), 10))

In [162]:
SVD_res['product_id'] = np.array(svd_res).flatten()

In [163]:
SVD_res.columns = ['user_id', 'product_id']

In [164]:
SVD_res['item_order'] = list(range(1, 11)) * test_data.user_id.nunique()

In [165]:
SVD_res

,user_id,product_id,item_order
0,116952,39877,1
1,116952,24852,2
2,116952,21137,3
3,116952,9839,4
4,116952,25931,5
...,...,...,...
2062085,184060,20119,6
2062086,184060,26620,7
2062087,184060,46676,8
2062088,184060,49044,9


In [167]:
SVD_res = SVD_res.merge(train_data.drop_duplicates(['user_id', 'product_id']), on=['user_id', 'product_id'], how='left')[['user_id', 'product_id', 'item_order', 'amount']]

In [168]:
SVD_res['amount'][SVD_res.amount.isna()] = 0

In [169]:
SVD_res.amount = SVD_res.amount / max(SVD_res.amount) * 3

In [166]:
v

Hitrate@10  0.64
MAP@10  0.428


In [170]:
SVD_res.groupby('user_id')['amount'].apply(list).apply(lambda x: ndcg(x, 10)).mean()

0.09147729585856695

- Hitrate@10  0.64
- MAP@10  0.428
- nDCG@10 0.091

## NN recommender

In [104]:
norm_df = train_data.copy()

In [105]:
from sklearn.preprocessing import MinMaxScaler
norm_df['amount'] = 1

In [106]:
u = norm_df.groupby('user_id').count()['product_id'].sort_values(ascending=True)[:20000].index

In [107]:
norm_df = norm_df[norm_df['user_id'].isin(u)]

In [108]:
user_map = {v: i for i, v in enumerate(sorted(norm_df.user_id.unique()))}
rev_user_map = {v: k for k, v in user_map.items()}

product_map = {v: i for i, v in enumerate(sorted(norm_df.product_id.unique()))}
rev_product_map = {v: k for k, v in user_map.items()}

In [109]:
norm_df['user_id'] = norm_df['user_id'].apply(lambda x: user_map[x])
norm_df['product_id'] = norm_df['product_id'].apply(lambda x: product_map[x])

In [17]:
# %%time
# negatives = pd.DataFrame(norm_df.groupby('order_id')['product_id'].apply(list).apply(lambda x:
#                                                                                          list(set(np.random.choice(
#                                                                                              norm_df.product_id.unique(),
#                                                                                              size=15)).difference(x))))
# negatives = negatives.reset_index()
# negatives['amount'] = -1

Wall time: 4min 7s


In [18]:
# negatives

,order_id,product_id,amount
0,7,"[1474, 20387, 18820, 17438, 19048, 12521, 1117...",-1
1,54,"[12995, 18277, 8487, 18312, 3757, 13903, 11345...",-1
2,100,"[5411, 19076, 3639, 8423, 20247, 12304, 7761, ...",-1
3,217,"[1636, 19621, 6664, 4618, 10252, 1773, 2157, 1...",-1
4,245,"[7872, 17316, 4901, 4741, 12292, 9195, 716, 10...",-1
...,...,...,...
74834,3420896,"[6145, 6499, 13860, 7589, 16389, 7844, 16746, ...",-1
74835,3420912,"[6978, 20547, 2820, 14151, 13479, 1193, 7272, ...",-1
74836,3420970,"[10722, 17031, 10600, 10984, 8855, 17100, 1278...",-1
74837,3421052,"[13633, 17797, 12296, 9066, 3053, 17389, 19065...",-1


In [19]:
# lst_col = 'product_id'
# negatives = pd.DataFrame({
#             col: np.repeat(negatives[col].values, negatives[lst_col].str.len())
#             for col in negatives.columns.drop(lst_col)}
#         ).assign(**{lst_col: np.concatenate(negatives[lst_col].values)})[negatives.columns]


In [20]:
# negatives

,order_id,product_id,amount
0,7,1474,-1
1,7,20387,-1
2,7,18820,-1
3,7,17438,-1
4,7,19048,-1
...,...,...,...
1122024,3421071,14169,-1
1122025,3421071,7256,-1
1122026,3421071,17564,-1
1122027,3421071,17981,-1


In [21]:
# norm_df1 = negatives.merge(norm_df[['user_id', 'order_id']].drop_duplicates(), on='order_id', how='left')

In [22]:
# norm_df = norm_df.append(norm_df1)

In [121]:
norm_test_df = test_data[test_data.user_id.isin(u)].copy()

In [122]:
norm_test_df['user_id'] = norm_test_df['user_id'].apply(lambda x: user_map[x])
norm_test_df['product_id'] = norm_test_df['product_id'].apply(lambda x: product_map[x] if x in product_map else -1)

In [123]:
norm_test_df = norm_test_df[norm_test_df['product_id'] != -1]

In [31]:
# # norm_df['user_id'] = norm_df['user_id'].apply(lambda x: user_map[x])
# norm_df['product_id'] = norm_df['product_id'].apply(lambda x: product_map[x])



# user_ids = norm_df["user_id"].unique().tolist()
# product_ids = norm_df["product_id"].unique().tolist()

# num_users = len(user_ids)
# num_products = len(product_ids)

In [147]:
# sorted(user_ids)

#### Validation is last order for each user in train dataset

In [36]:
last_orders = norm_df.order_id.isin(
            norm_df.sort_values(by=['user_id', 'order_number']).groupby('user_id').tail(1)['order_id'])
norm_train_df=norm_df[~last_orders]
norm_val_df=norm_df[last_orders]

In [37]:
norm_train_df[['user_id', 'product_id']].values

array([[ 9021, 13855],
       [ 9021,  7425],
       [ 9021, 19665],
       ...,
       [17797,  5498],
       [17797, 20091],
       [17797,  1919]], dtype=int64)

In [38]:
x, y = norm_train_df[['user_id', 'product_id']].values, norm_train_df['amount'].values
x_val, y_val = norm_val_df[['user_id', 'product_id']].values, norm_val_df['amount'].values

In [61]:
from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model

In [186]:
def NN_model(embedding_size = 200, classification = False):
    user = Input(name = 'user', shape = [1])
    product = Input(name = 'product', shape = [1])
    
    user_embedding = Embedding(name = 'user_embedding',
                               input_dim = num_users,
                               output_dim = embedding_size)(user)
    

    product_embedding = Embedding(name = 'product_embedding',
                               input_dim = num_products,
                               output_dim = embedding_size)(product)
    

    merged = Dot(name = 'dot_product', normalize = True, 
                 axes = 2)([user_embedding, product_embedding])
    

    merged = Reshape(target_shape = [1])(merged)
    
    out = Dense(1, activation = 'sigmoid')(merged)
    model = Model(inputs = [user, product], outputs = out)

    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', 
                  metrics = ['accuracy'])
    

    
    
    
    return model

In [187]:
model = NN_model()

In [188]:
norm_train_df['amount']

25649      1
25650      1
25651      1
25652      1
25653      1
          ..
1118204   -1
1118205   -1
1118206   -1
1118207   -1
1118208   -1
Name: amount, Length: 976358, dtype: int64

In [189]:
import numpy as np
import random
random.seed(42)

def generate_batch(pairs, n_positive = 10, negative_ratio = 1.0):
    """Generate batches of samples for training. 
       Random select positive samples
       from pairs and randomly select negatives."""
    
    # Create empty array to hold batch
    batch_size = n_positive * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    
    # Continue to yield samples
    while True:
        # Randomly choose positive examples
        for idx, (user_id, product_id) in enumerate(random.sample(pairs, n_positive)):
            batch[idx, :] = (user_id, product_id, 1)
        idx += 1
        
        # Add negative examples until reach batch size
        while idx < batch_size:
            
            # Random selection
            random_user = random.randrange(num_users)
            random_product = random.randrange(num_products)
            
            # Check to make sure this is not a positive example
            if (random_user, random_product) not in pairs:
                
                # Add to batch and increment index
                batch[idx, :] = (random_user, random_product, -1)
                idx += 1
                
        # Make sure to shuffle order
        np.random.shuffle(batch)
        yield {'user': batch[:, 0], 'product': batch[:, 1]}, batch[:, 2]

In [100]:
next(generate_batch(norm_df[['user_id', 'product_id']].values.tolist(), n_positive = 2, negative_ratio = 2))

({'user': array([  3603.,  12878.,  14907., 104290.,  14205., 162162.]),
  'product': array([17462., 11460.,  5726., 21554., 16608., 37046.])},
 array([-1., -1., -1.,  1., -1.,  1.]))

In [190]:
n_positive = 20

gen = generate_batch(norm_df[['user_id', 'product_id']].values.tolist(), n_positive, negative_ratio = 1)

# Train
h = model.fit_generator(gen, epochs = 10, 
                        steps_per_epoch = len(norm_df[['user_id', 'product_id']].values.tolist()) // n_positive)

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
11107/11107 [==============================] - 1725s 155ms/step - loss: -0.1574 - accuracy: 0.0406
Epoch 2/10
11107/11107 [==============================] - 1705s 154ms/step - loss: -6.0078 - accuracy: 0.1736
Epoch 3/10
11107/11107 [==============================] - 1699s 153ms/step - loss: -14.4087 - accuracy: 0.2433
Epoch 4/10
11107/11107 [==============================] - 1692s 152ms/step - loss: -21.7040 - accuracy: 0.2584- loss: -21.7032 - accura
Epoch 5/10
11107/11107 [==============================] - 1699s 153ms/step - loss: -28.8562 - accuracy: 0.2630
Epoch 6/10
11107/11107 [==============================] - 1694s 153ms/step - loss: -35.8061 - accuracy: 0.2634
Epoch 7/10
11107/11107 [==============================] - 1694s 153ms/step - loss: -42.9037 - accuracy: 0.2628
Epoch 8/10
11107/11107 [==============================] - 1695s 153ms/step - loss: -49.7052 - accuracy: 0.2604
Epoch 9/10
11107/11107 [==============================] - 1698s 153ms/step - loss: -56.59

In [191]:
pr_emb_layer = model.get_layer('product_embedding')
weights = pr_emb_layer.get_weights()[0]

In [192]:
weights =pd.DataFrame(weights)

In [193]:
from sklearn.neighbors import NearestNeighbors

n_model = NearestNeighbors(n_neighbors=10)
n_model.fit(weights)

NearestNeighbors(n_neighbors=10)

In [194]:
norm_test_df['user_id'].unique()

array([12739,  2709, 14120, ..., 16740,  9064, 10564], dtype=int64)

In [195]:
from tqdm import tqdm
keras_pred = []
for i in tqdm(norm_test_df['user_id'].unique()):
    keras_pred.append(n_model.kneighbors([weights.iloc[norm_df[norm_df['user_id'] == i]['product_id'].unique()].mean(axis=0)])[1][0])

100%|███████████████████████████████████████████████████████████████████████████| 19922/19922 [02:09<00:00, 154.19it/s]


In [196]:
keras_pred

[array([ 5486,  8296,  5385,  5598,  1729,    60,  3270, 13225, 15017,
          214], dtype=int64),
 array([ 3408, 10994, 18464, 14287, 10409, 17767, 15734,  1984,  9197,
        13253], dtype=int64),
 array([12798, 15727, 18208,  9009, 15134,  9188,  1438, 13403, 12366,
        16808], dtype=int64),
 array([16118, 17238,  3019, 11758, 19614, 20526, 13343,  4452, 12958,
         6387], dtype=int64),
 array([ 8878, 13823, 20246,  1642,  6179,  7386, 13947,  9377, 13736,
        13723], dtype=int64),
 array([18208,  1438, 12366,  7770,  9009, 14732, 17937,  6913, 13723,
         7386], dtype=int64),
 array([12798,  9009, 18208,  9188,  1438,  7386, 15727, 13723, 13403,
        12366], dtype=int64),
 array([16808, 18966, 18208,  1438, 19576, 17937,  2902, 15564,  3113,
        10906], dtype=int64),
 array([15017,  5385,  3270,  5598,  5486,  8296,  1729, 14459,    60,
         6471], dtype=int64),
 array([12798, 15134, 16808, 18208, 18966,  1438,  9009, 15727, 19576,
        13403], dtyp

In [197]:
NN_res = pd.DataFrame({
            'user_id': np.repeat(norm_test_df.user_id.unique(), 10),
            'product_id': np.array(pd.DataFrame(keras_pred)).flatten(),
            'item_order': list(range(1, 11)) * norm_test_df.user_id.unique().shape[0]
})

In [198]:
NN_res[NN_res.isna()] = 0

In [200]:
metrics(norm_test_df, NN_res)

In [184]:
NN_res = NN_res.merge(norm_train_df[['user_id', 'product_id', 'amount']], on=['user_id', 'product_id'])
NN_res.groupby('user_id')['amount'].apply(list).apply(lambda x: ndcg(x, 10)).mean()

- Hitrate@10  0.21
- MAP@10  0.103
- nDCG 0.038

## ALS recommender

In [89]:
import implicit

In [124]:
train_data

,user_id,product_id,order_id,amount,order_number
0,202279,33120,2,5,3
1,202279,33120,104690,5,8
2,202279,33120,132412,5,6
3,202279,33120,2808715,5,5
4,202279,33120,2894949,5,1
...,...,...,...,...,...
31653684,117521,13189,2858225,1,3
31653685,189140,47680,1666202,1,2
31653686,182854,34390,1699504,1,10
31653687,51836,31530,2666238,1,12


In [90]:
model = implicit.als.AlternatingLeastSquares(factors=50, random_state=42)

In [125]:
"""Fits transpose CF matrix to lib recommender"""
model.fit(CF.T.tocsr())

In [128]:
t = pd.Series(CF1[0].toarray()[0])

In [129]:
t[t != 0]

195      100
10253     81
10321      1
12421    100
13026      9
13170      4
14078      1
17116      1
25127     64
26081      4
26398      4
30439      1
35940      1
38913      1
39641      1
41771      1
46130      9
49215      4
dtype: int64

In [33]:
rev_item_map = {v: k for k, v in item_map.items()}

In [130]:
"""gets recs for each user from lib model"""

recs = []
for user in tqdm(test_data.user_id.unique()):
    ids = user_map[user]
    user_recs = model.recommend(ids, CF, N=10, filter_already_liked_items=False)
    user_recs = [rev_item_map[i[0]] for i in user_recs]
    recs.append(user_recs)

100%|█████████████████████████████████████████████████████████████████████████| 206209/206209 [03:33<00:00, 966.89it/s]


In [131]:
ALS_res = pd.DataFrame({
            'user_id': np.repeat(test_data.user_id.unique(), 10),
            'product_id': np.array(recs).flatten(),
            'item_order': list(range(1, 11)) * test_data.user_id.unique().shape[0]
        })

In [132]:
ALS_res

,user_id,product_id,item_order
0,116952,24852,1
1,116952,21137,2
2,116952,39877,3
3,116952,30391,4
4,116952,39275,5
...,...,...,...
2062085,184060,19348,6
2062086,184060,20119,7
2062087,184060,46979,8
2062088,184060,28204,9


In [133]:
test_data

,user_id,product_id,order_id
343,116952,33120,3302
344,116952,21137,3302
345,116952,49235,3302
346,116952,5077,3302
347,116952,25230,3302
...,...,...,...
33819051,204952,973,567548
33819083,102127,19833,607607
33819091,9798,43134,2570526
33819101,170084,28717,2839453


In [135]:
ALS_res = ALS_res.merge(train_data.drop_duplicates(['user_id', 'product_id']), on=['user_id', 'product_id'], how='left')[['user_id', 'product_id', 'item_order', 'amount']]

In [136]:
ALS_res['amount'][ALS_res.amount.isna()] = 0

metrics(test_data, ALS_res)

In [138]:
ALS_res.amount = ALS_res.amount / max(ALS_res.amount) * 3

In [139]:
ALS_res.groupby('user_id')['amount'].apply(list).apply(lambda x: ndcg(x, 10)).mean()

0.06043338214892533

- Hitrate@10  0.671
- MAP@10  0.353
- nDCG 0.060

## KNN on SVD embeddings

In [16]:
"""Uses svd embeddings of items to create neighbourhood (Recommender is  item-based)"""

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000, random_state=42)

svd_matrix = pd.DataFrame(svd.fit_transform(CF.transpose()))

In [18]:
from annoy import AnnoyIndex
from tqdm import tqdm

In [19]:
t = AnnoyIndex(1000, 'angular')

In [20]:
"""Creates items' neighbourhood"""
for i in tqdm(train_data.product_id.unique()):
    t.add_item(i, svd_matrix.loc[item_map[i]])

100%|██████████████████████████████████████████████████████████████████████████| 49668/49668 [00:21<00:00, 2313.16it/s]


In [21]:
t.build(10)

True

In [22]:
"""Gets the mean embedding of the products that user has ever bought"""
svd_matrix.loc[test_data[test_data.user_id == i].product_id.apply(lambda x: item_map[x]).unique()].mean(axis=0)

0     3351.495121
1     6973.871419
2    -1036.179614
3    -1404.534217
4     -213.842175
         ...     
95     -21.895512
96      41.767672
97     134.288380
98      26.092900
99     -84.206489
Length: 100, dtype: float64

In [23]:
"""Creates embedding for each user uses Annoy model to get recommendations of closest product to mean emb."""

KNN_res = pd.DataFrame(columns=['user_id', 'product_id', 'item_order'])
for i in tqdm(test_data.user_id.unique()):
    v = svd_matrix.loc[train_data[train_data.user_id == i].product_id.apply(lambda x: item_map[x]).unique()].mean(axis=0)
    KNN_res = KNN_res.append(pd.DataFrame({
            'user_id': np.repeat(i, 10),
            'product_id': t.get_nns_by_vector(v, 10),
            'item_order': list(range(1, 11))
        }))

100%|████████████████████████████████████████████████████████████████████████| 206209/206209 [5:25:17<00:00, 10.57it/s]


In [26]:
metrics(test_data, KNN_res)

In [27]:
KNN_res = KNN_res.merge(train_data.drop_duplicates(['user_id', 'product_id']), on=['user_id', 'product_id'], how='left')[['user_id', 'product_id', 'item_order', 'amount']]

In [28]:
KNN_res['amount'][KNN_res.amount.isna()] = 0

In [29]:
KNN_res.amount = KNN_res.amount / max(KNN_res.amount) * 3

In [36]:
KNN_res.groupby('user_id')['amount'].apply(list).apply(lambda x: ndcg(x, 10)).mean()

- HitRate@10     0.545
- MAP@10         0.305
- nDCG@10        0.041

<table>
    <tr>
        <th>Model</th>
        <th>Hitrate@10</th>
        <th>MAP@10</th>
        <th>nDCG@10</th>
    </tr>
    <tr>
        <td>Most popular rec</td>
        <td>0.457</td>
        <td>0.22</td>
        <td>0.034</td>
    </tr>
    <tr>
        <td>SVD rec</td>
        <td>0.64</td>
        <td>0.428</td>
        <td>0.091</td>
    </tr>
    <tr>
        <td>ALS rec</td>
        <td>0.671</td>
        <td>0.353</td>
        <td>0.060</td>
    </tr>
    <tr>
        <td>KNN on SVD rec</td>
        <td>0.545</td>
        <td> 0.305</td>
        <td>0.041</td>
    </tr>
    <tr>
        <td>NN rec</td>
        <td>0.21</td>
        <td> 0.0.103</td>
        <td>0.038</td>
    </tr>
</table>